In [1]:
# Adapted from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
# for kaggle satellite image classification dataset https://www.kaggle.com/mahmoudreda55/satellite-image-classification
# and then basic active learning was applied.

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from pytorch_finetuning_utils import train_model, train_model_given_numpy_arrays, initialize_model
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import sys
sys.path.append("..")
# sys.path.insert(0, '..')
from test_framework.model_interface import ModelInterface
from test_framework.tester import Tester
from utils.data_utils import get_kaggle_satellite_image_classification_dataset_as_numpy_arrays

%matplotlib inline

PyTorch Version:  1.10.0
Torchvision Version:  0.11.1


# parameters

In [2]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "../data/kaggle_satellite_image_classification"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 4

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 1#15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

# Flag for whether to perform data augmentation. When True, we perform random
#   operations (random crop, random flip, etc) to augment the data. When False,
#   we only perform deterministic preprocessing.
perform_data_augmentation = False

# Number of iterations of the active learning loop to perform
num_active_learning_iterations = 5

# Initialize model

In [3]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft.classifier)

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Conv2d(512, 4, kernel_size=(1, 1), stride=(1, 1))
  (2): ReLU(inplace=True)
  (3): AdaptiveAvgPool2d(output_size=(1, 1))
)


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load data

In [19]:
class ActiveLearningModel(ModelInterface):
    def __init__(self,model,name="no name provided",details="no details provided"):
        model.to(device)
        self.model = model
        self._name = name
        self._details = details
        # Gather the parameters to be optimized/updated in this run. If we are
        #  finetuning we will be updating all parameters. However, if we are
        #  doing feature extract method, we will only update the parameters
        #  that we have just initialized, i.e. the parameters with requires_grad
        #  is True.
        params_to_update = model.parameters()
        verbose = False
        if verbose:
            print("Params to learn:")
        if feature_extract:
            params_to_update = []
            for name,param in model.named_parameters():
                if param.requires_grad == True:
                    params_to_update.append(param)
                    if verbose:
                        print("\t",name)
        elif verbose:
            for name,param in model.named_parameters():
                if param.requires_grad == True:
                    print("\t",name)

        # Observe that all parameters are being optimized
        optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
        self._optimizer = optimizer_ft
        
        # store criterion
        self._criterion = nn.CrossEntropyLoss()

    def name(self) -> str:
        return self._name
    def details(self) -> str:
        return self._details
    def train(self, train_x:np.ndarray, train_y:np.ndarray) -> None:
        self.model = train_model_given_numpy_arrays(self.model, train_x, train_y, self._criterion, self._optimizer, num_epochs=num_epochs)
    def predict(self, test_x) -> np.ndarray:
        return self.model(torch.tensor(test_x).to(device)).cpu().detach().numpy()
    def query(self, unlabeled_data:np.ndarray, labeling_batch_size:int) -> np.ndarray:
        self.model.eval()
        softmax = nn.Softmax(dim=1)
        softmax_outputs = softmax(self.predict(unlabeled_data))
        max_softmax_vals, _ = torch.max(softmax_outputs, 1)
        max_softmax_vals_array = max_softmax_vals.cpu().detach().numpy()
        least_confident_indices = np.argpartition(max_softmax_vals_array,labeling_batch_size)[:labeling_batch_size]
        return least_confident_indices
    def query2(self, unlabeled_dataloader, labeling_batch_size:int) -> np.ndarray:
        self.model.eval()
        max_softmax_vals_list = []
        softmax = nn.Softmax(dim=1)
        for inputs, labels in unlabeled_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            softmax_outputs = softmax(self.model(inputs))
            max_softmax_vals, _ = torch.max(softmax_outputs, 1)
            max_softmax_vals_list += list(max_softmax_vals.cpu().detach().numpy())
        max_softmax_vals_array = np.array(max_softmax_vals_list)
        least_confident_indices = np.argpartition(max_softmax_vals_array,labeling_batch_size)[:labeling_batch_size]
        # print(sorted(max_softmax_vals_array.copy()))
        # print(max_softmax_vals_array[least_confident_indices])
        return least_confident_indices

In [15]:
x_data,y_data = get_kaggle_satellite_image_classification_dataset_as_numpy_arrays()

In [20]:
tester = Tester(x_data,y_data)
tester.TRAINING_EPOCHS = 2

In [21]:
active_learning_model = ActiveLearningModel(model_ft,model_name,str(model_ft.classifier))

In [22]:
tester.test_model(active_learning_model)

Test 0, Training Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0/0
----------


Test 0, Training Epoch 0:   0%|          | 0/10 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 10.77 GiB (GPU 0; 8.00 GiB total capacity; 2.85 GiB already allocated; 4.15 GiB free; 2.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
if perform_data_augmentation:
  transform = transforms.Compose([transforms.RandomResizedCrop(224), 
                                  transforms.ToTensor(),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomRotation(20),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
else:
  transform = transforms.Compose([transforms.CenterCrop(224),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = datasets.ImageFolder(data_dir, transform=transform)
# train_data, val_data, test_data = torch.utils.data.random_split(dataset, [3950, 1120,561])
unlabelled_train_data, labelled_train_data, val_data, test_data = torch.utils.data.random_split(dataset, [4113, 457, 500, 561])
print('Num unlabelled train images:', len(unlabelled_train_data))
print('Num labelled train images:  ', len(labelled_train_data))
print('Num val images:             ', len(val_data))
print('Num test images:            ', len(test_data))

In [ ]:
# define dataloader parameters
num_workers=0

# prepare data loaders
unlabelled_train_loader = torch.utils.data.DataLoader(unlabelled_train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=False)
labelled_train_loader = torch.utils.data.DataLoader(labelled_train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=False)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=False)
dataloaders_dict = {
    'unlabelled': unlabelled_train_loader,
    'train': labelled_train_loader,
    'val': valid_loader,
    'test': test_loader
}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Perform fine-tuning

In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
# model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

# active learning loop

In [ ]:
for i in range(num_active_learning_iterations):
    active_learning_model.train(dataloaders_dict)
    chosen_indices = active_learning_model.query(dataloaders_dict['unlabelled'],5)
    print(f"Iteration {i}: The model wants the following samples labelled: {chosen_indices}")
    negative_chosen_indices = [0] # TODO: make this the complement
    dataloaders_dict['unlabelled'] = torch.utils.data.Subset(dataloaders_dict['unlabelled'],negative_chosen_indices)

In [ ]:

class ActiveLearningModel(ModelInterface):
    def __init__(self,model,name="no name provided",details="no details provided"):
        self.model = model
        self.name = name
        self.details = details
    def name(self) -> str:
        return self.name
    def details(self) -> str:
        return self.details
    def train(self, train_x:np.ndarray, train_y:np.ndarray) -> None:
        self.model, hist = train_model(self.model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))
    def predict(self, test_x) -> np.ndarray:
        return self.model(test_x).cpu().detach().numpy()
    def query(self, unlabeled_dataloader, labeling_batch_size:int) -> np.ndarray:
        self.model.eval()
        max_softmax_vals_list = []
        softmax = nn.Softmax(dim=1)
        for inputs, labels in unlabeled_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            softmax_outputs = softmax(self.model(inputs))
            max_softmax_vals, _ = torch.max(softmax_outputs, 1)
            max_softmax_vals_list += list(max_softmax_vals.cpu().detach().numpy())
        max_softmax_vals_array = np.array(max_softmax_vals_list)
        least_confident_indices = np.argpartition(max_softmax_vals_array,labeling_batch_size)[:labeling_batch_size]
        # print(sorted(max_softmax_vals_array.copy()))
        # print(max_softmax_vals_array[least_confident_indices])
        return least_confident_indices